# Prise en charge des pneumopathies aigues communautaires

Ce notebook vise à établir un POC d'un outil qui pourrait être mis à disposition de professionnels de santé.  
**Fonctionnalité principale** : interface conversationnelle de questions-réponses basé sur un LLM qui peut répondre à des questions portant sur un document   
**Utilisateurs principaux** : professionnels de santé  
**Workflow** : l'utilisateur charge le document, l'outil analyse le document, l'utilisateur pose des questions sur le document, l'outil répond en fonction des données du document  
**Choix techniques** : l'outil proposé consiste en un LLM déjà entraîné, augmenté par le document fournit via un process RAG - Retrieval Augmented Generation.

## Environment setup

In [130]:
# Prerequisites
!pip install python-docx mistralai faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 61.4 MB/s eta 0:00:00


In [131]:
# Imports
import docx
import textwrap
from mistralai import Mistral
from google.colab import userdata
import numpy as np
import time
import faiss

In [61]:
# Mistral API Client
api_key= userdata.get("MISTRAL_API_KEY")
client = Mistral(api_key=api_key)

## Développement de l'outil et du modèle

### Traitement du document, extraction des informations

Le document consiste en des données textuelles à extraire : paragraphes, tableaux, images.  
NB : le cas de l'image ne sera pas traité dans un premier temps, on pourrait utiliser une librairie d'OCR pour en extraire des informations brutes, ou un LLM pour structurer les informations extraites (ex. image-to-text)

In [22]:
document = docx.Document("/content/Prise en charge des Pneumopathies aigues communautaires V2.docx")

In [72]:
def extract_data(document):

    doc_chunks = []

    # Extraction des textes dans l'ordre du document
    for item in document.iter_inner_content():

        # Paragraphes
        if isinstance(item, docx.text.paragraph.Paragraph):
            doc_chunks.append(item.text.strip())

        # Tables
        elif isinstance(item, docx.table.Table):
            for row in item.rows:
                row_data = ": ".join([cell.text for cell in row.cells])
                doc_chunks.append(row_data)

    # Nettoyage des éléments vides
    doc_chunks = [chunk for chunk in doc_chunks if chunk != ""]

    # Fusion des éléments
    doc_text = "\n".join(doc_chunks)

    return doc_text

In [73]:
doc_text = extract_data(document)

### Interprétation des textes en langage naturel via LLM

Le texte contient des données techniques "brutes" difficilement interpétables telles quelles. L'objectif est de les rendre plus compréhensibles via un LLM.

Le texte est maintenant prêt pour être passé dans un modèle LLM augmenté d'un RAG

### Paramétrage du RAG

In [52]:
# Découpage du document en chunks
def split_text(document, chunk_size=300):
    paragraphs = document.split("\n")
    chunks = []
    for paragraph in paragraphs:
        chunks.extend(textwrap.wrap(paragraph, chunk_size))
    return chunks

In [53]:
chunks = split_text(doc_text)

In [76]:
chunks

['PRISE EN CHARGE DES PNEUMOPATHIES AIGUES COMMUNAUTAIRES (PAC) SUPPOSEE D’ORIGINE BACTERIENNE',
 'Dont Pneumopathies graves, d’inhalation et légionellose',
 '* Le but de ce document est d’indiquer la prise en charge diagnostique et thérapeutique recommandée des PAC supposées d’origine bactérienne des patients hospitalisés (Ambulatoire non abordé)',
 '* Ne sont pas traité dans ces recommandations les patients avec DDB (mucoviscidose ou autres étiologies)',
 '* Mise à jour qui font suite aux dernières recommandations SPILF/SPLF sur les PAC publiées en 02/2025 dans le journal MMI Formation',
 'Point fort',
 'Critères de stabilité\xa0:  Apyrexie, stabilité tensionnelle, Pouls  100, FR  24, Saturation  à 90% en AA',
 'En cas de PAC non graves (hors USI) ET Obtention de l’ensemble des critères de stabilité à J3, un traitement de 3 jours est recommandé',
 'Si critères de stabilité obtenus à 5 jours, un traitement de 5 jours est recommandé',
 '(sans dépasser 7 jours dans les autres cas)',
 'A

In [86]:
# Génération d'un embedding pour un chunk
def get_one_embedding(chunk):
    embedding = client.embeddings.create(
        model = "mistral-embed",
        inputs = chunk
    )

    return embedding.data[0].embedding

In [128]:
# Limite d'utilisation API Mistral atteinte (vesion gratuite) : découpage en plusieurs parties pour générer les embeddings du doc d'entrée
# Warning - solution non scalable si documents plus longs : considérer un plan payant pour accès API ou gérer l'embedding en local

def get_all_embeddings(chunks, split_size=20):
    embeddings = []
    split_size = 30
    number_of_splits = len(chunks) // split_size + 1

    for i in range(number_of_splits):
        print("Chunk n°",i,"/",number_of_splits-1)
        split_of_chunks = chunks[split_size*i:min(split_size*(i+1),len(chunks))]
        embeddings_of_split = [get_one_embedding(chunk) for chunk in split_of_chunks]
        embeddings.extend(embeddings_of_split)

        time.sleep(60) # Attente pour limite API

    embeddings = np.array(embeddings)

    return embeddings

In [ ]:
embeddings = get_all_embeddings(chunks)

In [189]:
# Charger les embeddings du document dans une vector database FAISS
def create_index_from_embeddings(embeddings):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)

    return index

In [190]:
index = create_index_from_embeddings(embeddings)

Le LLM avec RAG est prêt pour être utilisé avec une question utilisateur !

### Générer une réponse augmentée du document

In [195]:
# L'utilisateur pose sa question
def get_user_prompt():
    question = input("Une question basée sur le document chargé : ")
    return question

In [196]:
question = get_user_prompt()

Une question basée sur le document chargé : Quels sont les critères majeurs de diagnostic de pneumopathie aigue grave ?


In [197]:
# Création de l'embedding associé à la question
question_embedding = np.array([get_one_embedding(question)])

In [198]:
# Retrouver les chunks les plus similaires dans l'index
def retrieve_chunk(index, question_embedding, top_k = 5):
    D, I = index.search(question_embedding, k=top_k)
    ind_chunks = I.tolist()[0]
    retrieved_chunks = [chunks[i] for i in range(len(chunks)) if i in ind_chunks]
    return " ".join(retrieved_chunks)

In [199]:
retrieved_chunk = retrieve_chunk(index, question_embedding)

In [204]:
# Combiner la question et la réponse dans un même prompt
def create_prompt(user_prompt, retrieved_chunk):
    prompt = f"""
    Contexte : {retrieved_chunk}
    -----------------------------------------
    Étant donné le context fourni ci-dessus, et sans utiliser de connaissance préalable, répond à la question suivante
    -----------------------------------------
    Question : {user_prompt}
    -----------------------------------------
    Format attendu : réponse complète, suivi des éléments du contexte qui t'ont permis de répondre
    -----------------------------------------
    Réponse :
    """

    return prompt

In [167]:
prompt = create_prompt(question, retrieved_chunk)

In [182]:
# Génération de la réponse par envoi du prompt au modèle decoder-only Mistral
def query_mistral(prompt, model="open-mistral-7b"):
    messages = [{
        "role": "user",
        "content": prompt
    }]

    chat_response = client.chat.complete(
        model=model,
        messages=messages
    )

    return chat_response.choices[0].message.content

In [185]:
response = query_mistral(prompt)

In [186]:
print(response)

Les critères majeurs de diagnostic de pneumopathie aigue grave sont :

* Détresse respiratoire nécessitant recours à la ventilation mécanique

Ces critères ont été définis dans le contexte fourni précédemment.


## Full pipeline

Fonction bout-en-bout pour un utilisateur qui a déjà chargé et fait analysé son document :    
- Input de la question de l'utilisateur
- Génération de l'embedding et recherche des chunks les plus simmilaires dans l'index
- Query au chat Mistral et affichage de la réponse

In [205]:
def ask_mistral_augmented(index, model="open-mistral-7b"):
    question = get_user_prompt()
    question_embedding = np.array([get_one_embedding(question)])
    retrieved_chunk = retrieve_chunk(index, question_embedding)
    prompt = create_prompt(question, retrieved_chunk)
    response = query_mistral(prompt)
    return response

In [210]:
response = ask_mistral_augmented(index)

Une question basée sur le document chargé : Quelle est la posologie à respecter pour l'amoxiciline en IV ? 


'La posologie à respecter pour l\'amoxicilline en IV est de 1 g par dose 3 fois par jour, ou 2 g par dose 3 fois par jour dans le cas de comorbidité. (Ce qui est mentionné dans le contexte en tant que : "Amoxicilline (IV ou PO): 1 g x3/j: 2 g x3/j").'